## Part 2: A Failure Mode for Linear Regression

In [1]:
import numpy as np
import pandas as pd

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline
import matplotlib.pyplot as plt

1. Create a feature matrix X with two columns and 100 rows. The first column should be an intercept column of all 1.0's, and the second should be randomly sampled from any distribution (a uniform is fine).

In [2]:
X = np.zeros(shape=(100, 2))

X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=100)

2. Create a target vector from a linear data generating process

In [3]:
# We'll use a smaller scale so the parameter estimates have
# smaller variance.
y = 1.0 + 2.0 * X[:, 1] + np.random.normal(scale=0.25, size=100)

3. Fit a linear regression to (X, y) data. Look at the fit coefficients (i.e. the parameter estimates in statistical language). Are they what you expect them to be? If you had fit the model to 1,000,000 data points, what would change about them?

In [4]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     702.7
Date:                Tue, 03 Nov 2020   Prob (F-statistic):           1.71e-46
Time:                        16:31:46   Log-Likelihood:                 11.833
No. Observations:                 100   AIC:                            -19.67
Df Residuals:                      98   BIC:                            -14.46
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0444      0.043     24.219      0.000       0.959       1.130
x1             1.9393      0.073     26.508      0.000       1.794       2.084
==============================================================================
Omnibus:                        1.136   Durbin-Watson:                   2.240
Prob(Omnibus):                  0.567   Jarque-Bera (JB):                1.094
Skew:                           0.248   Prob(JB):                        0.579
Kurtosis:                       2.873   Cond. No.                         4.31
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

If we use more data points then we expect the parameter esitmates to more precisely estimate the true values.  This relies on the data points being independently sampled from one another, and the statistical property that guarentees the better approximation is called **consistency**.

4. Create a new feature matrix X with three columns and 100 rows. Make the first two columns the same as your previous X, but make the third column a copy of the second column, i.e., X should have the same data in the second and third column.

In [5]:
X = np.zeros(shape=(100, 3))

X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=100)
X[:, 2] = X[:, 1]

y = 1.0 + 2.0 * X[:, 1] + np.random.normal(scale=0.25, size=100)

5. Fit a linear regression to the new (X, y) data (y should be the same as it was in the previous example). What happened?

In [6]:
model = sm.OLS(y, X)
results = model.fit(X, y)

/Users/jackbennetto/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:300: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if method == "pinv":
/Users/jackbennetto/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:315: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif method == "qr":


ValueError: method has to be "pinv" or "qr"

6. Hopefully you got an error, so there's something unfortunate going on here. Think about what you think the correct answer should be, what coefficients should the model return?

> It's not possible to say what the correct answer is in this situation.  We created the `y` vector using the equation:

> $$ y = 1 + 2 x_1 + \epsilon $$

> But in our current setup with two copies of the same predictor, this is exactly the same equation as:

> $$ y = 1 + 2 x_2 + \epsilon $$

> In fact, there is an infinite number of equivalent expressions:

> $$ y = 1 + x_1 + x_2 + \epsilon $$
> $$ y = 1 + 1.5 x_1 + 0.5 x_2 + \epsilon $$

> and so forth.  There's not really a way to say that any one of these is *better* than any other ones, they all give the same answer.

> In terms of algebra, we are looking for solutions the the following equation:

> $$ X^t X \beta = X^t y $$

> But the *rank* of the matrix $X^t X$ is two, since the columns of $X$ are linearly dependent.  This mean that this system of linear equations has an infinite number of solutions.  This is the source of the error, a *singluar* matrix is one without full rank.

7. Create a new feature matrix where one column is a multiple of another, and fit a linear regression again, what happened this time? How can you explain it?

In [7]:
X = np.zeros(shape=(100, 3))

X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=100)
X[:, 2] = 2 * X[:, 1]

y = 1.0 + 2.0 * X[:, 1] + np.random.normal(scale=0.25, size=100)

In [8]:
model = sm.OLS(y, X)
results = model.fit(X, y)

ValueError: method has to be "pinv" or "qr"

> We've got more or less the same issue as before. This time the > following equations are all equivalent:

> $$ y = 1 + 2 x_1 + \epsilon $$
> $$ y = 1 + x_2 + \epsilon $$
> $$ y = 1 + x_1 + 0.5 x_2 + \epsilon $$

> So we again have an issue with the rank of the matrix $X^t X$.

8. Create one last feature matrix where one column is a linear combination of two or more other columns. Fit a linear regression using it. What happened this time? Can you explain it?

In [9]:
X = np.zeros(shape=(100, 4))

X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=100)
X[:, 2] = np.random.uniform(size=100)
X[:, 3] = 2 * X[:, 1] - 3 * X[:, 2]

y = (1.0
     + 2.0 * X[:, 1]
     - 1.0 * X[:, 2]
     + np.random.normal(scale=0.25, size=100))

In [10]:
model = sm.OLS(y, X)
results = model.fit()

> This time the model fit...

In [11]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     339.5
Date:                Tue, 03 Nov 2020   Prob (F-statistic):           1.59e-44
Time:                        16:31:52   Log-Likelihood:                0.13183
No. Observations:                 100   AIC:                             5.736
Df Residuals:                      97   BIC:                             13.55
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9915      0.064     15.452      0.000       0.864       1.119
x1             0.9651      0.067     14.508      0.000       0.833       1.097
x2             0.4837      0.045     10.713      0.000       0.394       0.573
x3             0.4791      0.022     21.296      0.000       0.434       0.524
==============================================================================
Omnibus:                        9.277   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.010   Jarque-Bera (JB):                9.188
Skew:                          -0.640   Prob(JB):                       0.0101
Kurtosis:                       3.752   Cond. No.                     1.81e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.99e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

> But the coefficients are way off. Note the condition number ("Cond. No.") is huge, indicating multicollinearity. In such a case the coefficients can't really be trusted.

> This is a sign that we have some problems in our regression.  While before, our regression just failed because we had an exact linear dependency, here floating point error saved us from having an exact problem, but the linear dependency in our matrix has led to badly estimated parameters.

> Notice that the parameter estimates returned are very bad estimates of the truth.  This is a common situation when our matrix has either an exact linear dependency, or an approximate one.  The results from these regression should not be trusted!  It's indicating to us that we could remove one of our predictors without suffering:

In [12]:
X_small = X[:, [0, 1, 2]]

model = sm.OLS(y, X_small)
results = model.fit()

In [13]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     339.5
Date:                Tue, 03 Nov 2020   Prob (F-statistic):           1.59e-44
Time:                        16:31:54   Log-Likelihood:                0.13183
No. Observations:                 100   AIC:                             5.736
Df Residuals:                      97   BIC:                             13.55
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9915      0.064     15.452      0.000       0.864       1.119
x1             1.9232      0.079     24.215      0.000       1.766       2.081
x2            -0.9536      0.088    -10.834      0.000      -1.128      -0.779
==============================================================================
Omnibus:                        9.277   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.010   Jarque-Bera (JB):                9.188
Skew:                          -0.640   Prob(JB):                       0.0101
Kurtosis:                       3.752   Cond. No.                         5.26
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

> That's much better.

9. Hopefully you've seen a few linear regressions fail at this point. Why did they fail? What is the failure mode for linear regression?

> Linear regressions fail when the columns of the predictor matrix $X$ are linearly independent, which leads to having multiple (an infinite number of) equally good solutions.  This was explicit in our first few examples.

> In our final example, we again had a linear dependency in $X$, but the computer did not catch it exactly.  Instead, it was indicated the large condition number.  This tends to happen when our $X$ matrix either contains an exact or approximate linear independency.